In [ ]:
# If you are in a fresh environment (e.g., Colab / new venv), uncomment:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install diffusers transformers accelerate safetensors
!pip install pillow kagglehub opencv-python


In [6]:
import shutil
import cv2
import json
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
from pathlib import Path
import random
import torch
from diffusers import StableDiffusionPipeline


In [7]:
def white_to_alpha(img: Image.Image, threshold: int = 240) -> Image.Image:
    img = img.convert("RGBA")
    data = np.array(img)
    r, g, b, a = data.T
    white = (r > threshold) & (g > threshold) & (b > threshold)
    data[..., 3][white.T] = 0
    return Image.fromarray(data)

def load_prompt_files(prompt_file, neg_file, default_prompt="", default_negative=""):
    prompt, negative = default_prompt, default_negative
    if prompt_file and Path(prompt_file).exists():
        prompt = Path(prompt_file).read_text(encoding="utf-8")
    if neg_file and Path(neg_file).exists():
        negative = Path(neg_file).read_text(encoding="utf-8")
    return prompt, negative

def init_sd_pipeline(model_id: str, device: str = "cuda", dtype=torch.float16):
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=dtype)
    pipe = pipe.to(device)
    pipe.safety_checker = None
    try:
        pipe.enable_attention_slicing()
    except Exception:
        pass
    return pipe

def generate_textures_from_cfg(cfg: dict, pipe: StableDiffusionPipeline):
    textures_dir = Path(cfg["textures_dir"])
    textures_count = cfg.get("textures_count", 12)
    texture_size = cfg.get("texture_size", 512)
    prompt_path = cfg.get("prompt_path")
    negative_path = cfg.get("negative_path")
    prompt, negative = load_prompt_files(prompt_path, negative_path, "", "")
    textures_dir.mkdir(parents=True, exist_ok=True)
    for i in range(textures_count):
        img = pipe(
            prompt=prompt,
            negative_prompt=negative,
            height=texture_size,
            width=texture_size,
            guidance_scale=7
        ).images[0]
        img = white_to_alpha(img)
        img.save(textures_dir / f"texture_{i:04d}.png")
    print(f"Textures generated: {textures_dir}")


In [8]:
def load_config(config_path="config.json"):
    with open(config_path, "r") as f:
        return json.load(f)

def get_mask_params(level: str, cfg):
    if level not in cfg["mask_config"]:
        raise ValueError(f"Unknown contamination level: {level}")
    return cfg["mask_config"][level]

def generate_contamination_mask_multiscale(size,rng: np.random.Generator,mask_cfg: dict):
    """
    mask_cfg is expected to come directly from:
    CFG["mask_config"][level]["mask"]
    """
    w, h = size
    mask = Image.new("L", (w, h), 0)
    draw = ImageDraw.Draw(mask)

    spot_specs = mask_cfg["spot_specs"]
    base_blur = mask_cfg.get("base_blur", 0)
    edge_blur = mask_cfg.get("edge_blur", 0)
    add_streaks = mask_cfg.get("add_streaks", False)

    # --- blob generation ---
    for (
        count,
        min_r,
        max_r,
        sub_n,
        sub_rf,
        inten_min,
        inten_max,
    ) in spot_specs:

        for _ in range(int(count)):
            main_r = int(rng.integers(min_r, max_r + 1))

            cx = int(rng.integers(main_r, max(main_r + 1, w - main_r)))
            cy = int(rng.integers(main_r, max(main_r + 1, h - main_r)))

            intensity = int(rng.integers(inten_min, inten_max + 1))

            for _ in range(int(sub_n)):
                ox = int(rng.integers(-main_r, main_r + 1))
                oy = int(rng.integers(-main_r, main_r + 1))

                sub_r = max(
                    1,
                    int(main_r * sub_rf * float(rng.uniform(0.5, 1.6)))
                )

                sx = cx + ox
                sy = cy + oy

                x1 = max(0, sx - sub_r)
                y1 = max(0, sy - sub_r)
                x2 = min(w, sx + sub_r)
                y2 = min(h, sy + sub_r)

                if x1 < x2 and y1 < y2:
                    draw.ellipse((x1, y1, x2, y2), fill=intensity)

    # --- streaks ---
    if add_streaks:
        n_streaks = int(rng.integers(3, 10))
        for _ in range(n_streaks):
            x1 = int(rng.integers(0, w))
            y1 = int(rng.integers(0, h))

            length = int(rng.integers(int(0.10 * w), int(0.35 * w)))
            angle = float(rng.uniform(-0.6, 0.6))
            thickness = int(rng.integers(1, 3))
            intensity = int(rng.integers(30, 90))

            x2 = int(np.clip(x1 + length * np.cos(angle), 0, w - 1))
            y2 = int(np.clip(y1 + length * np.sin(angle), 0, h - 1))

            draw.line((x1, y1, x2, y2), fill=intensity, width=thickness)

    # --- blur ---
    if base_blur > 0:
        mask = mask.filter(ImageFilter.GaussianBlur(base_blur))
    if edge_blur > 0:
        mask = mask.filter(ImageFilter.GaussianBlur(edge_blur))

    return mask


In [14]:
def apply_refined_soiling_v2(
    clean_img: Image.Image,
    texture_img: Image.Image,
    rng: np.random.Generator,
    level_cfg: dict,
    return_mask: bool = True,
):
    """
    level_cfg is expected to be:
    CFG["mask_config"][level]
    """

    clean_img = clean_img.convert("RGB")
    w, h = clean_img.size

    # --- generate mask (config-driven) ---
    mask_raw = generate_contamination_mask_multiscale(
        size=(w, h),
        rng=rng,
        mask_cfg=level_cfg["mask"]
    )

    mask_np = np.array(mask_raw).astype(np.float32) / 255.0

    # --- edge softness ---
    edge_softness = level_cfg.get("edge_soft", 0)
    if edge_softness > 0:
        k = int(edge_softness) * 2 + 1
        mask_np = cv2.GaussianBlur(mask_np, (k, k), 0)

    # --- alpha ---
    alpha_strength = level_cfg.get("alpha", 1.0)
    mask_np = np.clip(mask_np * float(alpha_strength), 0.0, 1.0)

    # --- texture ---
    tex = texture_img.resize((w, h)).convert("RGBA")
    tex_np = np.array(tex).astype(np.float32)
    tex_rgb = tex_np[..., :3]
    tex_a = tex_np[..., 3] / 255.0

    m = np.clip(mask_np * np.clip(tex_a * 0.85, 0.0, 1.0), 0.0, 1.0)
    m3 = m[..., None]

    # --- blend ---
    I = np.array(clean_img).astype(np.float32)
    dirty = (1.0 - m3) * I + m3 * tex_rgb

    # --- darken ---
    darken_strength = level_cfg.get("darken", 0.0)
    if darken_strength > 0:
        dirty *= (1.0 - float(darken_strength) * m3)

    # --- blur ---
    blur_strength = level_cfg.get("blur", 0)
    if blur_strength > 0:
        k = int(blur_strength) * 2 + 1
        blurred = cv2.GaussianBlur(dirty, (k, k), 0)
        dirty = (1.0 - m3) * dirty + m3 * blurred

    # --- haze ---
    haze_strength = level_cfg.get("haze", 0.0)
    if haze_strength > 0:
        dirty = (
            (1.0 - float(haze_strength) * m3) * dirty
            + (float(haze_strength) * m3) * 255.0
        )

    dirty_img = Image.fromarray(
        np.uint8(np.clip(dirty, 0, 255))
    )

    if not return_mask:
        return dirty_img

    mask_img = Image.fromarray(
        np.uint8(np.clip(m * 255.0, 0, 255)),
        mode="L"
    )

    return dirty_img, mask_img

def build_dirty_kitti_subset(
    img_dir: Path,
    lbl_dir: Path,
    n_samples: int,
    overwrite_output: bool,
    make_all_presets_per_image: bool,
    CFG: dict,
    out_root: Path,
    img_subset: list[Path] = None,
):
    rng = np.random.default_rng()

    # -----------------------------
    # Read from CFG
    # -----------------------------
    textures_dir = Path(CFG["textures_dir"])
    dirty_dir_rel = Path(CFG["dirty_dir"])
    clean_dir_rel = Path(CFG["clean_dir"])
    masks_dir_rel = Path(CFG["masks_dir"])

    save_clean = CFG.get("save_clean", False)
    save_masks = CFG.get("save_masks", False)

    # -----------------------------
    # Output dirs inside out_root
    # -----------------------------
    dirty_dir_path = out_root / dirty_dir_rel
    clean_dir_path = out_root / clean_dir_rel
    masks_dir_path = out_root / masks_dir_rel

    if overwrite_output:
        for d in [dirty_dir_path, clean_dir_path, masks_dir_path]:
            if d.exists():
                shutil.rmtree(d)

    dirty_dir_path.mkdir(parents=True, exist_ok=True)
    if save_clean:
        clean_dir_path.mkdir(parents=True, exist_ok=True)
    if save_masks:
        masks_dir_path.mkdir(parents=True, exist_ok=True)

    # Labels folder (always)
    out_lbl_dir = dirty_dir_path.parent / "label_2"
    out_lbl_dir.mkdir(parents=True, exist_ok=True)

    # -----------------------------
    # Load data
    # -----------------------------
    if img_subset is not None:
        chosen = img_subset
    else:
        all_imgs = sorted(Path(img_dir).glob("*.png"))
        if not all_imgs:
            raise FileNotFoundError(f"No images found in {img_dir}")
        chosen = random.sample(all_imgs, k=min(n_samples, len(all_imgs)))

    textures = sorted(textures_dir.glob("*.png"))
    if not textures:
        raise FileNotFoundError(
            f"No gen_textures found in {textures_dir}. Generate or add PNG gen_textures first."
        )

    missing_labels = 0
    saved = 0

    # -----------------------------
    # Main loop
    # -----------------------------
    for img_path in chosen:
        stem = img_path.stem
        lbl_path = Path(lbl_dir) / f"{stem}.txt"

        if not lbl_path.exists():
            missing_labels += 1
            continue

        clean_img = Image.open(img_path)
        tex_img = Image.open(random.choice(textures))

        # Save clean image if needed
        if save_clean:
            clean_img.convert("RGB").save(clean_dir_path / f"{stem}.png", format="PNG")

        # Select severities
        if make_all_presets_per_image:
            severities = list(CFG["mask_config"].keys())
        else:
            severities = [random.choice(list(CFG["mask_config"].keys()))]

        for sev in severities:
            level_cfg = CFG["mask_config"][sev]

            # --- FIX: handle single or tuple return ---
            result = apply_refined_soiling_v2(
                clean_img=clean_img,
                texture_img=tex_img,
                rng=rng,
                level_cfg=level_cfg,
                return_mask=save_masks,
            )
            if save_masks:
                dirty_img, mask_img = result
            else:
                dirty_img = result
                mask_img = None

            out_name = f"{stem}_{sev}" if make_all_presets_per_image else stem

            # Save dirty image
            dirty_img.save(dirty_dir_path / f"{out_name}.png", format="PNG")

            # Save mask if needed
            if save_masks and mask_img is not None:
                mask_img.save(masks_dir_path / f"{out_name}.png", format="PNG")

            # Copy label
            shutil.copy2(lbl_path, out_lbl_dir / f"{out_name}.txt")
            saved += 1

    print(f"Done. Saved subset to: {dirty_dir_path.parent}")
    print(f"Missing labels skipped: {missing_labels}")
    print(f"Dirty images saved: {saved}")



In [21]:
img_dir = Path(r"C:\Users\yuval\.cache\kagglehub\datasets\klemenko\kitti-dataset\versions\1\data_object_image_2\testing\image_2")
lbl_dir = Path(r"C:\Users\yuval\.cache\kagglehub\datasets\klemenko\kitti-dataset\versions\1\data_object_label_2\training\label_2")
out_root = Path(r"C:\Users\yuval\PycharmProjects\GenAI_Project\dirty_imgs")
MAKE_ALL_PRESETS_PER_IMAGE = False
mud_cfg = load_config("gen_textures/mud/config.json")
water_cfg = load_config("gen_textures/water/config.json")

In [11]:
pipe = init_sd_pipeline("runwayml/stable-diffusion-v1-5")
generate_textures_from_cfg(mud_cfg, pipe)
generate_textures_from_cfg(water_cfg, pipe)

100%|██████████| 50/50 [00:34<00:00,  1.46it/s]


Textures generated: C:\Users\yuval\PycharmProjects\GenAI_Project\textures\mud


 10%|█         | 5/50 [00:04<00:39,  1.15it/s]


KeyboardInterrupt: 

In [22]:
all_imgs = sorted(img_dir.glob("*.png"))
half = len(all_imgs) // 2
mud_imgs = all_imgs[:half]
water_imgs = all_imgs[half:]

build_dirty_kitti_subset(
    img_dir=img_dir,
    lbl_dir=lbl_dir,
    n_samples=len(mud_imgs),
    overwrite_output=True,
    make_all_presets_per_image=MAKE_ALL_PRESETS_PER_IMAGE,
    CFG=mud_cfg,
    out_root=out_root / "mud",
    img_subset=mud_imgs[:50]
)

build_dirty_kitti_subset(
    img_dir=img_dir,
    lbl_dir=lbl_dir,
    n_samples=len(water_imgs),
    overwrite_output=True,
    make_all_presets_per_image=MAKE_ALL_PRESETS_PER_IMAGE,
    CFG=water_cfg,
    out_root=out_root / "water",
    img_subset=water_imgs[:50]
)


Done. Saved subset to: C:\Users\yuval\PycharmProjects\GenAI_Project\dirty_imgs\mud
Missing labels skipped: 0
Dirty images saved: 50
Done. Saved subset to: C:\Users\yuval\PycharmProjects\GenAI_Project\dirty_imgs\water
Missing labels skipped: 0
Dirty images saved: 50
